In [5]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import sys
sys.path.insert(0, '/home/ektov-av/python35-libs/lib/python3.5/site-packages/') 
sys.path.insert(1,'../pyutils/pretty-print-confusion-matrix/')

In [6]:
import argparse
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import os
from src import utils
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from confusion_matrix_pretty_print import pretty_plot_confusion_matrix, plot_confusion_matrix_from_data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from tqdm import tqdm

%matplotlib inline

In [18]:
torch.__version__

'1.0.0'

In [7]:
path2csv = Path.joinpath(Path(os.getcwd()), 'data','topics_val.csv')
df = pd.read_csv(path2csv, encoding='utf8', na_filter=False)
df.fillna('', inplace=True)

In [ ]:
df[df.Sentiment==1]

In [8]:
use_cuda = torch.cuda.is_available()

def predict(args):
    model = torch.load(args.model)
    processed_input = utils.preprocess_input(args)
    processed_input = torch.tensor(processed_input)
#     processed_input = processed_input.unsqueeze(0)
    if use_cuda:
        processed_input = processed_input.to('cuda')
        model = model.to('cuda')
    prediction = model(processed_input)
    probabilities = F.softmax(prediction, dim=1)
    proba = probabilities.cpu().detach().numpy()
    pred_cls  = np.argmax(proba,axis=-1)
    return pred_cls

In [9]:
# indx = 3000
newcorpus=df.SentimentText[:].tolist()
label=df.Sentiment[:].tolist()

In [10]:
len(newcorpus)

8080

In [11]:
model_files = glob.glob(os.path.join(os.getcwd(),'models','*.pth'))

In [12]:
model_files[-1]

'/home/ektov-av/notebooks/character_based_cnn/models/char_cnn_epoch_6_lr_0.0025_loss_0.4051_acc_0.8198.pth'

In [13]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        'Testing a pretrained Character Based CNN for text classification')
    parser.add_argument('--model', type=str, default=model_files[-1], 
                                   help='path for pre-trained model')

    parser.add_argument('--text', nargs="+",
                        default=newcorpus, help='corpora list')
    parser.add_argument('--steps', nargs="+", default=['remove_html','lower'])
    parser.add_argument('--batch_size', type=int, default=1000)

    # arguments needed for the predicition
    parser.add_argument('--alphabet', type=str,
                        default="""абвгдеёжзиклмнопрстюуфхчшщэюяьъabcdefghijklmnopqrstuvwxyz0123456789""")
    parser.add_argument('--number_of_characters', type=int, default=67)
    parser.add_argument('--extra_characters', type=str, default='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    parser.add_argument('--max_length', type=int, default=600)

In [14]:
def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
args = parser.parse_args(argv)

In [15]:
torch.nn.Module.dump_patches = True

In [16]:
model = torch.load(args.model)

## Prediction

In [ ]:
batch_size = args.batch_size
corpora_text = args.text

batch_params = {"batch_size": batch_size,
                "shuffle": False,
                "num_workers": 4}

batch_generator = DataLoader(corpora_text, **batch_params)
prediction_lst = []

for iter, batch in tqdm(enumerate(batch_generator), total=len(batch_generator)):
        args.text = batch
        prediction = predict(args)
        prediction_lst.extend(prediction)

In [ ]:
# prediction = predict(args)
# print('input text: {}'.format(args.text))
# print('-'*40)
# print('true_class : {}'.format(label))
# print('prediction : {}'.format(prediction))

In [ ]:
# prediction = predict(args)

In [ ]:
cm = confusion_matrix(label,prediction)
fig = print_confusion_matrix(cm,class_names=[0,1])

In [ ]:
plot_confusion_matrix_from_data(label,prediction,columns=[0,1])